Import Libraries

In [97]:
from pyvis.network import Network
import networkx as nx
import random

Create Helper Functions

In [ ]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''

    # base condition, stop at node
    if random.random() < probStop:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    authorID = authors[-1]
    newNeighbors = network.neighbors(authorID)
    for neighbor in network.neighbors(authorID):
        nData = network.get_edge_data(authorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    if len(probs):
        coauthorID = random.choice(probs)
    # randomly select author from all authors because it is a new author
    else:
        otherNodes = list(network.nodes())
        otherNodes.pop(authorID)
        # edge case if there is only one node in the network
        if not len(otherNodes):
            return
        coauthorID = random.choice(otherNodes)
        network.add_edge(authorID, coauthorID, weight=0, width=1)

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(authorID, coauthorID)["weight"] + 1
        #network.update(edges=[ (authorID, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (authorID, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)

    # can't go back to initial authors

    # new author must have one collaborator

def create():
    return

Define initial parameters

In [99]:
# define time steps
timeSteps = 50

# Probabilities
# probability that you stop at the given node
pw = 0.5

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [100]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    allNodes = list(network.nodes())
    authors = [random.choice(allNodes)]

    # with probability, add new author to network
    if random.random() < pw:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor
        otherNodes = list(network.nodes())
        otherNodes.pop(author)
        coauthorID = random.choice(otherNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        #update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    probStop = 0.3
    createPaper(network, authors, probStop)

    # add authors as members to this topic

    # add emergence of new topic if field multidiscipiplinary (i.e. the union of the fields)

    # do you make it the author of the paper?

Display Network

In [101]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
nt
nt.show('testModel.html')